##### Check if the datasets are balanced

In [2]:
import os
import pandas as pd

# Load fashion-related data (positive samples)
fashion_df = pd.read_csv('/home/disk1/red_disk1/Multimodal_MKT/topics_filtered.csv')

# Extract the 'keyword' column which contains fashion-related keywords
fashion_samples = fashion_df['keyword group']

# Directory containing non-fashion-related samples
neg_dir = "/home/disk1/red_disk1/Multimodal_MKT/non_fashion_texts"

# Initialize an empty list to store non-fashion samples
non_fashion_samples = []

# Recursively iterate through all subdirectories and files in non_fashion_texts directory
for root, dirs, files in os.walk(neg_dir):
    for file in files:
        # Read only text files
        if file.startswith('wiki'):
            file_path = os.path.join(root, file)
            with open(file_path, 'r', encoding='utf-8') as f:
                # Add each line as a sample
                lines = f.readlines()
                non_fashion_samples.extend([line.strip() for line in lines if line.strip()])

# Convert non-fashion samples to a pandas Series for consistency
non_fashion_samples = pd.Series(non_fashion_samples)

# Step 2: Count the number of samples in each dataset
num_fashion_samples = fashion_samples.shape[0]
num_non_fashion_samples = non_fashion_samples.shape[0]

# Print the results
print(f"Number of fashion-related samples (positive): {num_fashion_samples}")
print(f"Number of non-fashion-related samples (negative): {num_non_fashion_samples}")

# Step 3: Check if they are balanced
if num_fashion_samples == num_non_fashion_samples:
    print("The datasets are balanced.")
else:
    print(f"The datasets are not balanced. Difference: {abs(num_fashion_samples - num_non_fashion_samples)}")


Number of fashion-related samples (positive): 52242
Number of non-fashion-related samples (negative): 304611
The datasets are not balanced. Difference: 252369


#### Downsampling:
- randomly reducing the number of samples from the majority class (non-fashion-related samples) to match the number of samples in the minority class (fashion-related samples).

In [3]:
import random

# If the non-fashion dataset is larger, downsample it
if num_non_fashion_samples > num_fashion_samples:
    # Randomly select `num_fashion_samples` non-fashion samples
    downsampled_non_fashion_samples = non_fashion_samples.sample(n=num_fashion_samples, random_state=42)
else:
    downsampled_non_fashion_samples = non_fashion_samples

# Print the number of samples after downsampling
print(f"Number of downsampled non-fashion-related samples: {downsampled_non_fashion_samples.shape[0]}")


Number of downsampled non-fashion-related samples: 52242


#### Step 1: Read Data Files
- read positive samples
- read negative samples
- combine the datasets
#### Step 2: Load pre-trained sentence Transformer Model
- load text2vec-large-chinese model
- generate embeddings for each word
#### Step 3: Get embeddings for the dataset
- apply the embeddings function
- convert labels to tensor
#### Step 4: Train-test split
- split the data
- create PyTorh TensorDataset and DataLoader

In [4]:
import torch
from torch import nn
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from torch.utils.data import DataLoader, TensorDataset
import torch.nn.functional as F
from sentence_transformers import SentenceTransformer
from tqdm import tqdm
import os

# Step 1: Read Positive Samples from 'topics_filtered.csv'
# Load the fashion lexicon (positive samples)
pos_file = "/home/disk1/red_disk1/fashion/topics_filtered.csv"
positive_samples_df = pd.read_csv(pos_file)
positive_samples = positive_samples_df['keyword group'].dropna().tolist()  # Extract keywords column

# Assign label 1 to positive samples
positive_samples = pd.DataFrame(positive_samples, columns=['word'])
positive_samples['label'] = 1

# Step 2: Read Negative Samples from non_fashion_texts
def read_negative_samples(directory):
    all_lines = []
    for foldername in os.listdir(directory):
        folder_path = os.path.join(directory, foldername)
        if os.path.isdir(folder_path):
            for filename in os.listdir(folder_path):
                file_path = os.path.join(folder_path, filename)
                with open(file_path, 'r', encoding='utf-8') as file:
                    all_lines.extend([line.strip() for line in file if line.strip() != ""])
    return all_lines

neg_dir = "/home/disk1/red_disk1/Multimodal_MKT/non_fashion_texts"
negative_samples_list = read_negative_samples(neg_dir)

# Assign label 0 to negative samples
negative_samples = pd.DataFrame(negative_samples_list, columns=['word'])
negative_samples['label'] = 0

# Step 3: Combine Positive and Negative Samples
data = pd.concat([positive_samples, negative_samples], ignore_index=True)

# Shuffle the dataset
data = data.sample(frac=1).reset_index(drop=True)

# Print the shape and format of the data
print(f"Combined Data Shape: {data.shape}")
print(f"Data Format Example:\n{data.head()}")

# Step 4: Load text2vec-large-chinese model
model_path = "/home/disk1/red_disk1/fashion/text2vec-large-chinese/models--GanymedeNil--text2vec-large-chinese"
text2vec_model = SentenceTransformer(model_path)

# Function to get embeddings from text2vec-large-chinese
def get_text2vec_embeddings(texts, model):
    print("Generating embeddings...")
    embeddings = []
    for i in tqdm(range(0, len(texts), 32), desc="Embedding Progress"):
        batch_texts = texts[i:i+32].tolist()
        batch_embeddings = model.encode(batch_texts, convert_to_tensor=True)
        embeddings.append(batch_embeddings)
    return torch.cat(embeddings, dim=0)

# Step 5: Get embeddings for the dataset
embeddings = get_text2vec_embeddings(data['word'], text2vec_model)

# Print the shape and format of the embeddings
print(f"Embeddings Shape: {embeddings.shape}")
print(f"Embedding Example:\n{embeddings[0]}")

# Convert labels to tensor
labels = torch.tensor(data['label'].values)

# Print the labels format
print(f"Labels Shape: {labels.shape}")
print(f"Labels Example:\n{labels[:5]}")

# Step 6: Train-Test Split
train_embeddings, val_embeddings, train_labels, val_labels = train_test_split(
    embeddings, labels, test_size=0.2, random_state=42
)

# Print the split data shapes
print(f"Train Embeddings Shape: {train_embeddings.shape}, Train Labels Shape: {train_labels.shape}")
print(f"Val Embeddings Shape: {val_embeddings.shape}, Val Labels Shape: {val_labels.shape}")

# Create TensorDataset and DataLoader for training and validation
train_dataset = TensorDataset(train_embeddings, train_labels)
val_dataset = TensorDataset(val_embeddings, val_labels)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32)

# Print the data loader batch size
print(f"Train Loader Batch Size: {len(train_loader)} batches")
print(f"Val Loader Batch Size: {len(val_loader)} batches")


No sentence-transformers model found with name /home/disk1/red_disk1/fashion/text2vec-large-chinese/models--GanymedeNil--text2vec-large-chinese. Creating a new one with mean pooling.


Combined Data Shape: (356855, 2)
Data Format Example:
                                                word  label
0  由百獸戰隊牙吠連者起，在全員唱完名後會喊出一句標語，再接戰隊的隊名，而要準備開戰前，紅戰士都...      0
1                                                歷史.      0
2                                            傳統建築語彙.      0
3        石洞口发电厂目前共有6台发电机组，总装机容量240万千瓦。在组织结构上，分为三个实体。      0
4                                               LNTU      0


/home/diandian/.local/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Generating embeddings...


Embedding Progress: 100%|██████████| 11152/11152 [17:51<00:00, 10.41it/s]

Embeddings Shape: torch.Size([356855, 1024])
Embedding Example:
tensor([-0.8630,  0.6928, -0.4863,  ...,  0.3087, -0.2302,  0.3473],
       device='cuda:0')
Labels Shape: torch.Size([356855])
Labels Example:
tensor([0, 0, 0, 0, 0])
Train Embeddings Shape: torch.Size([285484, 1024]), Train Labels Shape: torch.Size([285484])
Val Embeddings Shape: torch.Size([71371, 1024]), Val Labels Shape: torch.Size([71371])
Train Loader Batch Size: 8922 batches
Val Loader Batch Size: 2231 batches


### Define and train a ResNet-based binary classification model
1. Model Definition: ResNet-Based Binary Classifier
2. Training Loop
3. Validation Loop
4. Training and Evaluation

#### Summary
- Model Setup: It initializes a ResNet18 model tailored for binary classification using text embeddings as inputs.
- Training and Validation: The code implements a standard training loop with backpropagation and uses validation to track performance.
- Performance Metrics: Accuracy, precision, recall, and F1 score are calculated to evaluate the binary classification model.

In [5]:
import torch.optim as optim
from torch import nn
from tqdm import tqdm
from torchvision.models import resnet18
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

# Step 7: Define ResNet-Based Binary Classification Model
class ResNetBinaryClassifier(nn.Module):
    def __init__(self, embedding_dim=1024):  # Adjust embedding_dim based on text2vec output (1024)
        super(ResNetBinaryClassifier, self).__init__()
        # Use pre-trained ResNet18, but replace the first conv layer to accept text2vec embeddings
        self.resnet = resnet18(pretrained=False)
        self.resnet.fc = nn.Linear(self.resnet.fc.in_features, 2)  # Binary classification
        
        # Define a simple MLP layer to map embeddings to ResNet input size
        self.embedding_to_resnet = nn.Sequential(
            nn.Linear(embedding_dim, 512),  # Match the expected ResNet input size
            nn.ReLU(),
            nn.Linear(512, 512),  # Match the expected ResNet input size after projection
        )
        
        # Adding a Conv layer to convert 1D to a 2D feature map
        self.embedding_conv = nn.Conv2d(1, 3, kernel_size=(1, 1))

        # Adaptive pooling layer to transform the spatial dimensions to (7, 7)
        self.adaptive_pool = nn.AdaptiveAvgPool2d((7, 7))

    def forward(self, x):
        x = self.embedding_to_resnet(x)  # Project text2vec embeddings to match ResNet input size
        x = x.unsqueeze(1).unsqueeze(2)  # Reshape to fit Conv2D (batch_size, channels, height, width)
        x = self.embedding_conv(x)  # Use conv layer to convert 1D feature to 2D feature map
        x = self.adaptive_pool(x)  # Apply adaptive pooling to resize to (7, 7)
        x = self.resnet(x)
        return x

# Initialize the model with the correct embedding dimension (1024 as per the output shape)
model = ResNetBinaryClassifier(embedding_dim=1024)

# Step 8: Define Training Parameters
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

# Training function
def train(model, train_loader, criterion, optimizer, device):
    model.train()
    total_loss = 0
    for batch in tqdm(train_loader, desc="Training Progress"):
        inputs, labels = batch
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(inputs)
        
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
    
    return total_loss / len(train_loader)

# Validation function
def evaluate(model, val_loader, criterion, device):
    model.eval()
    total_loss = 0
    all_preds, all_labels = [], []
    with torch.no_grad():
        for batch in tqdm(val_loader, desc="Validation Progress"):
            inputs, labels = batch
            inputs, labels = inputs.to(device), labels.to(device)
            
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            total_loss += loss.item()
            
            preds = torch.argmax(outputs, dim=1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    
    accuracy = accuracy_score(all_labels, all_preds)
    precision, recall, f1, _ = precision_recall_fscore_support(all_labels, all_preds, average='binary')
    return total_loss / len(val_loader), accuracy, precision, recall, f1

# Step 9: Training Loop
epochs = 5
for epoch in range(epochs):
    print(f"Epoch {epoch+1}/{epochs}")
    train_loss = train(model, train_loader, criterion, optimizer, device)
    val_loss, accuracy, precision, recall, f1 = evaluate(model, val_loader, criterion, device)
    
    print(f"Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}, Recall: {recall:.4f}, F1 Score: {f1:.4f}")

# Step 10: Final Evaluation
val_loss, accuracy, precision, recall, f1 = evaluate(model, val_loader, criterion, device)
print(f"Final Results: Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1 Score: {f1:.4f}")


/home/diandian/.local/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/diandian/.local/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Epoch 1/5


Validation Progress: 100%|██████████| 2231/2231 [00:08<00:00, 270.69it/s]


Train Loss: 0.0862, Val Loss: 0.0649, Accuracy: 0.9743
Precision: 0.9378, Recall: 0.8820, F1 Score: 0.9090
Epoch 2/5


Validation Progress: 100%|██████████| 2231/2231 [00:08<00:00, 270.66it/s]


...

Train Loss: 0.0322, Val Loss: 0.0469, Accuracy: 0.9820
Precision: 0.9435, Recall: 0.9321, F1 Score: 0.9378
Epoch 5/5


Validation Progress: 100%|██████████| 2231/2231 [00:08<00:00, 271.77it/s]


Train Loss: 0.0261, Val Loss: 0.0480, Accuracy: 0.9815
Precision: 0.9386, Recall: 0.9336, F1 Score: 0.9361


Validation Progress: 100%|██████████| 2231/2231 [00:06<00:00, 324.81it/s]


Final Results: Accuracy: 0.9815, Precision: 0.9386, Recall: 0.9336, F1 Score: 0.9361


In [6]:
# Step 11: Save the trained model

# Save model and tokenizer to the desired directory
model_save_path = "/home/disk1/red_disk1/fashion/tfashion"

# Make the directory if it doesn't exist
os.makedirs(model_save_path, exist_ok=True)

# Save the entire model
torch.save(model.state_dict(), os.path.join(model_save_path, "tfashion.pth"))

# Optionally, save additional model metadata if needed (e.g., optimizer state, epoch)
checkpoint = {
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'epoch': epochs,
}
torch.save(checkpoint, os.path.join(model_save_path, "tfashion_checkpoint.pth"))

print(f"Model saved at: {model_save_path}")


Model saved at: /home/disk1/red_disk1/fashion/tfashion


#### Prepare post data
- Combine 'post_title' and 'post_content' into 'post_text'
- Clean 'post_text'

In [7]:
import pandas as pd

# Load the data
df = pd.read_csv('/home/disk1/red_disk1/fashion/poster_test_fashion_nlpclean.csv')
# df = pd.read_csv('/home/disk1/red_disk1/poster_9305.csv')

# Combine 'post_title' and 'post_content' into 'post_text'
df['post_text'] = df['post_title'].fillna('') + ' ' + df['post_content'].fillna('')

# Drop 'post_title' and 'post_content' columns
df = df.drop(columns=['post_title', 'post_content', 'post_tag'])

# Save the updated DataFrame
# df.to_csv('/home/disk1/red_disk1/Multimodal_MKT/poster_9305_combined.csv', index=False)
df.to_csv('/home/disk1/red_disk1/fashion/poster_test_fashion_nlpclean_combined.csv', index=False)

In [8]:
import emoji
import re

# Load stopwords from the provided file
with open('/home/disk1/red_disk1/fashion/stopwords_cn.txt', 'r', encoding='utf-8') as f:
    stopwords = set(f.read().splitlines())

# Function for text cleaning
def clean_text(text, stopwords):
    # Convert emojis to text
    text = emoji.demojize(text)
    
    # Remove specific patterns
    text = re.sub(r'- 小红书,,', '', text)  # Removing "- 小红书,,"
    text = re.sub(r'小红书', '', text)  # Explicitly remove "小红书"
    text = re.sub(r',,\d{2}-\d{2},,', '', text)  # Removing patterns like ",,XX-XX,,"
    text = re.sub(r'#', ' ', text)  # Replace '#' with a space
    text = re.sub(r'\s+', '', text)  # This will remove all whitespace characters

    # Remove digits
    text = re.sub(r'\d+', '', text)
    
    # Remove special characters
    cleaned_text = ''.join(char for char in text if char.isalnum() or char.isspace())
    
    # Remove stopwords (word-based removal)
    cleaned_text = ' '.join([word for word in cleaned_text.split() if word not in stopwords])
    
    return cleaned_text

# Apply cleaning function to 'post_text'
df['cleaned_post_text'] = df['post_text'].apply(lambda x: clean_text(str(x), stopwords))

# Remove duplicates
df = df.drop_duplicates()

# Save the cleaned DataFrame
df.to_csv('/home/disk1/red_disk1/fashion/post_cleaned.csv', index=False)


#### (optional) Test the model tfashion
- Step 1: Load the test dataset.
- Step 2: Generate embeddings for each word in the test dataset using text2vec-large-chinese.
- Step 3: Use a pre-trained ResNet-based classifier model to predict whether each word is related to fashion or not.
- Step 4: Save the predictions alongside the original data into a CSV file.

In [2]:
import torch
import pandas as pd
from sentence_transformers import SentenceTransformer
from torch.utils.data import DataLoader, TensorDataset
from tqdm import tqdm
from torch import nn
import os

# Load the pre-trained SentenceTransformer model (text2vec-large-chinese)
model_path = "/home/disk1/red_disk1/fashion/text2vec-large-chinese/models--GanymedeNil--text2vec-large-chinese"
text2vec_model = SentenceTransformer(model_path)

# Function to get embeddings from text2vec-large-chinese
def get_text2vec_embeddings(texts, model):
    embeddings = []
    for i in tqdm(range(0, len(texts), 32), desc="Generating Embeddings"):
        batch_texts = texts[i:i+32].tolist()
        batch_embeddings = model.encode(batch_texts, convert_to_tensor=True)
        embeddings.append(batch_embeddings)
    return torch.cat(embeddings, dim=0)

# Step 1: Load the test dataset
test_file = "/home/disk1/red_disk1/fashion/post_cleaned.csv"  # Update this path if necessary
test_data = pd.read_csv(test_file)

# Get the words column from the test dataset
test_words = test_data['cleaned_post_text']

# Step 2: Get embeddings for the test words
test_embeddings = get_text2vec_embeddings(test_words, text2vec_model)

# Step 3: Create a DataLoader for the test data
test_dataset = TensorDataset(test_embeddings)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Step 4: Load the trained tfashion model
class ResNetBinaryClassifier(nn.Module):
    def __init__(self, embedding_dim=1024):
        super(ResNetBinaryClassifier, self).__init__()
        self.resnet = resnet18(pretrained=False)
        self.resnet.fc = nn.Linear(self.resnet.fc.in_features, 2)
        self.embedding_to_resnet = nn.Sequential(
            nn.Linear(embedding_dim, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
        )
        self.embedding_conv = nn.Conv2d(1, 3, kernel_size=(1, 1))
        self.adaptive_pool = nn.AdaptiveAvgPool2d((7, 7))

    def forward(self, x):
        x = self.embedding_to_resnet(x)
        x = x.unsqueeze(1).unsqueeze(2)
        x = self.embedding_conv(x)
        x = self.adaptive_pool(x)
        x = self.resnet(x)
        return x

# Load the trained tfashion model from your specified path
model_save_path = "/home/disk1/red_disk1/fashion/tfashion/tfashion.pth"
model = ResNetBinaryClassifier(embedding_dim=1024)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.load_state_dict(torch.load(model_save_path, map_location=device))
model.to(device)
model.eval()

# Step 5: Perform inference on the test dataset
output_labels = []
with torch.no_grad():
    for batch in tqdm(test_loader, desc="Testing Progress"):
        inputs = batch[0].to(device)
        outputs = model(inputs)
        preds = torch.argmax(outputs, dim=1)
        output_labels.extend(preds.cpu().numpy())

# Step 6: Save the results
# Add the output_labels as a new column to the original test_data DataFrame
test_data['output_label'] = output_labels

# Save the updated DataFrame to a new CSV file
output_file = "/home/disk1/red_disk1/fashion/test-output.csv"  
test_data.to_csv(output_file, index=False)

print(f"Results saved to {output_file}")


No sentence-transformers model found with name /home/disk1/red_disk1/fashion/text2vec-large-chinese. Creating a new one with mean pooling.


OSError: Error no file named pytorch_model.bin, model.safetensors, tf_model.h5, model.ckpt.index or flax_model.msgpack found in directory /home/disk1/red_disk1/fashion/text2vec-large-chinese.

#### Apply the model
1. Tokenization: tokenize the post_text content
2. Word Embedding: pass each word (instead of the entire post) through the text2vec-large-chinese model to generate embeddings for each word
3. Classification: pass embeddings through the tfashion model to classify whether each word is fashion-related or not
4. Filter out any words that are present in the stopwords set before applying the model classification
5. Filter out single-character words
6. Results: The output will be a classification for each word

In [9]:
import torch
import pandas as pd
import jieba
from tqdm import tqdm
from sentence_transformers import SentenceTransformer
from torch.utils.data import DataLoader, TensorDataset
from torch import nn
import os
import re

# Step 1: Load the stopwords file
stopwords_file = "/home/disk1/red_disk1/fashion/stopwords_cn.txt"
with open(stopwords_file, 'r', encoding='utf-8') as f:
    stopwords = set([line.strip() for line in f.readlines()])

# Step 2: Load the pre-trained SentenceTransformer model (text2vec-large-chinese)
model_path = "/home/disk1/red_disk1/fashion/text2vec-large-chinese/models--GanymedeNil--text2vec-large-chinese"
text2vec_model = SentenceTransformer(model_path)

# Step 3: Define ResNetBinaryClassifier
class ResNetBinaryClassifier(nn.Module):
    def __init__(self, embedding_dim=1024):
        super(ResNetBinaryClassifier, self).__init__()
        self.resnet = resnet18(pretrained=False)
        self.resnet.fc = nn.Linear(self.resnet.fc.in_features, 2)
        self.embedding_to_resnet = nn.Sequential(
            nn.Linear(embedding_dim, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
        )
        self.embedding_conv = nn.Conv2d(1, 3, kernel_size=(1, 1))
        self.adaptive_pool = nn.AdaptiveAvgPool2d((7, 7))

    def forward(self, x):
        x = self.embedding_to_resnet(x)
        x = x.unsqueeze(1).unsqueeze(2)
        x = self.embedding_conv(x)
        x = self.adaptive_pool(x)
        x = self.resnet(x)
        return x

# Load the model
model_file = "/home/disk1/red_disk1/fashion/tfashion/tfashion.pth"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = ResNetBinaryClassifier(embedding_dim=1024)
model.load_state_dict(torch.load(model_file, map_location=device))
model.to(device)
model.eval()

# Step 4: Load the CSV file containing post_text
csv_file = "/home/disk1/red_disk1/fashion/post_cleaned.csv"
df = pd.read_csv(csv_file)

# Step 5: Tokenize the post_text content into words and remove stopwords
def tokenize_text(text):
    words = list(jieba.cut(text))
    # Retain English words using regex (keeping words with ASCII characters)
    english_words = re.findall(r'[a-zA-Z]+', text)
    # Filter out stopwords and single-character Chinese words
    filtered_words = [word for word in words if word not in stopwords and len(word) > 1]
    return filtered_words + english_words  # Keep both filtered Chinese and English words

df['tokenized_text'] = df['cleaned_post_text'].apply(tokenize_text)

# Step 6: Get embeddings for each word in the tokenized_text column and classify them
def get_text2vec_embeddings(words, model):
    embeddings = model.encode(words, convert_to_tensor=True)
    return embeddings

filtered_keywords = []

for idx, row in tqdm(df.iterrows(), total=len(df), desc="Processing Rows"):
    words = row['tokenized_text']
    
    # Get embeddings for the words
    embeddings = get_text2vec_embeddings(words, text2vec_model).to(device)
    
    # Make predictions using the pre-trained ResNet binary classifier
    with torch.no_grad():
        outputs = model(embeddings)
        preds = torch.argmax(outputs, dim=1).cpu().numpy()
    
    # Filter words where the model predicts label 1 (fashion-related), and keep English words
    filtered_words = [word for word, label in zip(words, preds) if label == 1 or re.match(r'[a-zA-Z]+', word)]
    filtered_keywords.append(filtered_words)

# Step 7: Add the new column filtered_keywords and save the DataFrame
df['filtered_keywords2'] = filtered_keywords

# Save the updated DataFrame to a new CSV file
output_file = "/home/disk1/red_disk1/fashion/test_filtered.csv"
df.to_csv(output_file, index=False)

print(f"Filtered results saved to {output_file}")


No sentence-transformers model found with name /home/disk1/red_disk1/fashion/text2vec-large-chinese/models--GanymedeNil--text2vec-large-chinese. Creating a new one with mean pooling.


/home/diandian/.local/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
/home/diandian/.local/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/diandian/.local/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
Building prefix dict from the default dictionary ...
Loading model from

Filtered results saved to /home/disk1/red_disk1/fashion/test_filtered.csv


##### Check the output

In [10]:
import pandas as pd

# Step 1: 读取文件
csv_file = "/home/disk1/red_disk1/fashion/test_filtered.csv"
df = pd.read_csv(csv_file)

# Step 2: 随机采样30行
sampled_df = df.sample(n=30, random_state=42)  # 随机采样30行

# Step 3: 去重函数，保持顺序不变
def remove_duplicates(word_list):
    seen = set()
    return [x for x in word_list if not (x in seen or seen.add(x))]

# Step 4: 打印输出其中的 summary_cleaned 列和 filter_label 列，去掉 filter_label 中的重复单词
for i, row in sampled_df.iterrows():
    summary_cleaned = eval(row['tokenized_text'])  # 转换为列表
    filter_label = eval(row['filtered_keywords2'])  # 转换为列表
    filter_label_unique = remove_duplicates(filter_label)  # 去重
    
    print(f"Example {i+1}:")
    print(f"summary_cleaned: {summary_cleaned}")
    print(f"filter_label: {filter_label_unique}")
    print()  # 空行用于美观输出

Example 12997:
summary_cleaned: ['尊嘟', 'redexclamationmark', '富贵', '小姐姐', '尝试', '温柔', '气质', '姐姐', '富家', '千金', '秋冬', '温柔', '慵懒', '毛衣', '慵懒', '风穿', '高级', '感穿', 'redexclamationmark', 'look']
filter_label: ['尊嘟', 'redexclamationmark', '富贵', '小姐姐', '尝试', '温柔', '气质', '姐姐', '富家', '千金', '秋冬', '慵懒', '毛衣', '风穿', '高级', '感穿', 'look']

Example 11585:
summary_cleaned: ['温柔', '白月光', '感觉', '试试', '仙气', '裙子', '中式', '民国', '温柔', '连衣裙', '国风', '针织', '中式', '套装']
filter_label: ['温柔', '白月光', '感觉', '试试', '仙气', '裙子', '中式', '民国', '连衣裙', '国风', '针织', '套装']

Example 11226:
summary_cleaned: ['我先', '微胖', '女孩', '微胖']
filter_label: ['我先', '微胖', '女孩']

Example 13552:
summary_cleaned: ['redapple', '跟着', '模特', 'keycapkeycap', '复古', '百褶裙', '太好', '身高', 'cm', '体重', '斤斤', '小腿', 'cm', '大腿', 'cm', '腰围', 'cm', '肚围', 'cmredapple', '身材', '我太爱', '这件', '短裙', '几年', '尝试', '百褶裙', '没想到', '感觉', '不错', '哈哈哈', '材质', '材质', '质感', '苹果', '身材', '苹果', '小个子', '短裙', '短裙', '格子裙', '百褶裙', '百褶裙', '百褶裙', '格子', '百褶裙', '复古', '百褶裙', '复古', '格子', '半裙', '秋季', 

In [13]:
import ast
import pandas as pd

# Step 1: Load the CSV file
csv_file = "/home/disk1/red_disk1/fashion/test_filtered.csv"
df = pd.read_csv(csv_file)

# Step 2: Sample 30 rows randomly
# sampled_df = df.sample(n=30, random_state=42)
sampled_df = df.sample(n=30)

# Step 3: Function to remove duplicates while maintaining order
def remove_duplicates(word_list):
    seen = set()
    return [x for x in word_list if not (x in seen or seen.add(x))]

# Step 4: Iterate over sampled rows and process the content
for i, row in sampled_df.iterrows():
    try:
        # Keep the original cleaned_post_text as is for display
        summary_cleaned_original = row['cleaned_post_text']
        
        # Attempt to safely convert filtered_keywords to list, if possible
        if isinstance(row['filtered_keywords2'], str):
            try:
                filter_label = ast.literal_eval(row['filtered_keywords2'])  # Safely convert to list
            except (ValueError, SyntaxError):
                # Treat it as a simple string if not a valid list
                filter_label = row['filtered_keywords2'].split()
        else:
            filter_label = row['filtered_keywords2']
    
    except (ValueError, SyntaxError):
        # Log problematic rows
        print(f"Error parsing row {i}, skipping this row.")
        continue
    
    # Remove duplicates from filtered_keywords
    filter_label_unique = remove_duplicates(filter_label)
    
    # Output the original text and processed keywords
    print(f"Example {i+1}:")
    print(f"Original post_text: {summary_cleaned_original}")
    print(f"Filtered keywords: {filter_label_unique}")
    print()  # Empty line for better output formatting


Example 4542:
Original post_text: rU领修身短袖plus宽松牛仔裤太显瘦啦link上短下长遮胯显瘦裤子我的ootdOOTD穿搭分享时尚百搭超会穿企划ootd每日穿搭显瘦穿搭正肩t恤修身短袖
Filtered keywords: ['rU', '修身', '短袖', '宽松', '牛仔裤', '太显', 'link', '下长', '显瘦', '裤子', 'ootdOOTD', '分享', '时尚', '百搭', '超会', '企划', 'ootd', '每日', 'plus', 't']

Example 9801:
Original post_text: 一眼心动有多少次都爱的鱼尾连衣裙dress这条人鱼尾连衣裙也太显身材了这个颜色也很绝一点也不艳俗下摆我真的超喜欢层层褶皱走起路来很美日常穿搭连衣裙显瘦连衣裙Ootd今天穿什么香连衣裙显瘦穿搭夏天穿裙子针织裙温柔穿搭我的日常吊带裙鱼尾裙每日穿搭笔记灵感
Filtered keywords: ['一眼', '心动', '鱼尾', '连衣裙', 'dress', '这条', '太显', '身材', '颜色', '一点', '不艳俗', '下摆', '真的', '喜欢', '层层', '褶皱', '走起路来', '日常', '显瘦', 'Ootd', '夏天', '裙子', '针织', '温柔', '吊带裙', '每日', '笔记', '灵感']

Example 11898:
Original post_text: 好辣我说裙子好显身材这条感觉很适合过节穿哈哈哈日常穿搭每日穿搭我的日常穿搭
Filtered keywords: ['好辣', '裙子', '好显', '身材', '感觉', '适合', '哈哈哈', '日常', '每日']

Example 513:
Original post_text: 纽约拥堵费省moneybag方案这算一劳永逸嘛omggrimacingface纽约成为全美首个推行拥堵费的城市而且一转眼就要开始正式实行了meltingfacecalendar开始时间月日起motorway收费路段进入曼岛中城街及以南就必须支付moneybag私家车价格kitchenknife每辆车每天仅收一次moneybag网约车kitchenknife次按次收费alarmclock